In [1]:
import django
django.setup()

from crypto_track.models import CryptoCandle, PyTrends, Bank, SignalSimulation, Simulation
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [45]:
candle_data = list(CryptoCandle.objects.all().values().order_by('-period_start_timestamp'))
next_candle = ""
for i, cd in enumerate(candle_data):
    candle_data[i]['next_candle_period_close'] = next_candle
    next_candle = candle_data[i]['period_close']

In [19]:
candle_data = candle_data.sort(key='period_start_timestamp')

TypeError: 'str' object is not callable

In [46]:
df = pd.DataFrame(candle_data)
df.head()

,crypto_traded,currency_quoted,data_source,id,next_candle_period_close,period_close,period_high,period_interval,period_low,period_open,period_start_timestamp,period_volume,search_trend_id,update_timestamp
0,BTC,USD,Nomics https://api.nomics.com/v1/candles,19089,,6544.0724600000,6815.6190200000,1d,6441.5660000000,6482.6548500000,2018-11-07T00:00:00Z,822208473.0000000000,2018-11-07,2018-11-07 19:54:17.385457+00:00
1,BTC,USD,Nomics https://api.nomics.com/v1/candles,19088,6544.0724600000,6471.7850800000,6839.6380400000,1d,6375.0495800000,6426.9369100000,2018-11-06T00:00:00Z,1088715834.0000000000,2018-11-06,2018-11-07 19:54:17.385457+00:00
2,BTC,USD,Nomics https://api.nomics.com/v1/candles,19087,6471.7850800000,6423.6074500000,6714.2151300000,1d,6359.9270600000,6446.7592800000,2018-11-05T00:00:00Z,760242069.0000000000,2018-11-05,2018-11-07 19:54:17.385457+00:00
3,BTC,USD,Nomics https://api.nomics.com/v1/candles,19086,6423.6074500000,6456.3840100000,6755.5497900000,1d,6299.7255300000,6362.6646700000,2018-11-04T00:00:00Z,929200549.0000000000,2018-11-04,2018-11-07 19:54:17.385457+00:00
4,BTC,USD,Nomics https://api.nomics.com/v1/candles,19085,6456.3840100000,6373.1894200000,6793.4570900000,1d,6300.6000000000,6404.7765300000,2018-11-03T00:00:00Z,570044456.0000000000,2018-11-03,2018-11-07 19:54:17.385457+00:00


In [49]:
import numpy as np
df = df[(len(df.next_candle_period_close) > 0)]
df_convert = df.replace( '',np.nan, regex=True)
df['period_close'] = df.period_close.astype(float)
df['next_candle_period_close'] = df.next_candle_period_close.astype(float)

df.dtypes

KeyError: True

In [21]:
# Create target object and call it y
y = df.next_candle_period_close

In [22]:
features = ['period_close', 'period_high', 'period_low', 'period_open', 'period_volume']
X = df[features]

In [24]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [25]:
# Specify Model
btc_model = DecisionTreeRegressor(random_state=1)
# Fit Model
btc_model.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [26]:
# Make validation predictions and calculate mean absolute error
val_predictions = btc_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

ValueError: could not convert string to float: 